In [2]:
import os

from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

#api key(추가해서 쓰시오)
import settings
import pickle

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.chains import RetrievalQA, HypotheticalDocumentEmbedder

from langchain.prompts import PromptTemplate

os.environ["OPENAI_API_KEY"] = settings.openai_api_key

# directory = os.path.dirname(__file__)
# os.chdir(directory)

def _device_check() : 
    ''' for check cuda availability '''
    import torch
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # torch.backends.mps.is_available()
    return device


In [3]:
#llm
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1)

#embedding config
embedding = SentenceTransformerEmbeddings(
    model_name="BM-K/KoSimCSE-roberta-multitask", 
    model_kwargs={'device':_device_check()}, 
    encode_kwargs={'normalize_embeddings':True},
    )

## embedding config - HyDE
hyde_prompt_template = """ 
    Write a passage in Korean to answer the #question in detail.

    #question : {question}
    #passage : ...

"""

hyde_prompt_complete = PromptTemplate(input_variables=["question"], template= hyde_prompt_template)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# llm_chain = prompt | llm
llm_chain = LLMChain(
    llm=llm, 
    prompt=hyde_prompt_complete,
)

# HyDEembeddings = hyde_llm | embedding
HyDEembeddings = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain,
    base_embeddings=embedding,
)

#get vectorstore *HyDE Embedding
vectorstore = Chroma(collection_name="vector_db", persist_directory="./chroma_storage", embedding_function=HyDEembeddings)

# print(vectorstore.similarity_search("국가장학금", k=3,))

#get document from pickle(use as documents in bm25_retriever)
# with open('./document.pkl', 'rb') as file :
    # documents = pickle.load(file)

documents = """# 근로장려금

### 대상

근로·사업·종교인 소득이 있는 가구 중 소득·재산요건을 충족하는 가구
* '22.12.31. 현재 가구원(배우자·18세 미만 부양자녀·70세 이상 직계존속) 유무와 배우자의 총급여액 등에 따라 가구를 구분하며, 자녀장려금은 부양자녀가 있어야 합니다.

| 구분       | 설명 |
|------------|------|
| 단독가구   | 배우자, 부양자녀 및 부양 직계존속이 모두 없는 가구 |
| 홑벌이 가구 | 배우자, 18세 미만 부양자녀 또는 부양 직계존속이 있는 가구(단, 배우자의 총급여액 등이 300만 원 미만인 경우에 한함) |
| 맞벌이 가구 | 신청인과 배우자 각각의 총급여액 등이 300만 원 이상인 가구 |

- 재산요건: 2022년 6월 1일 기준 재산 합계액 2.4억 원 미만인 가구
- 소득요건: 가구 구성에 따라 2022년 총소득 기준금액이 아래 금액 미만인 가구

| 구분       | 단독가구     | 홑벌이 가구   | 맞벌이 가구   |
|------------|--------------|--------------|--------------|
| 총소득기준금액 | 2,200만 원 | 3,200만 원 | 3,800만 원 |
| 근로장려금 최대지급액 | 165만 원 | 285만 원 | 330만 원 |

※ 재산 합계액이 1억 7,000만 원 이상 2억 4,000만 원 미만일 경우 장려금의 50% 지급

### 내용

일은 하지만 소득이 적어 생활이 어려운 근로자, 사업자(전문직 제외) 또는 종교인 가구에 대하여 근로장려금을 지급함으로써 근로 또는 사업을 장려하고 실질소득을 지원

### 방법

① (원칙) 매년 5월 신청 → 9월 지급<br>
※ 기한 후 신청 시(6.1~11.30) 10% 감액 지급<br>
② (근로소득자) 당해연도 반기별 지급 선택 가능<br>
- (상반기 소득분) 9.1∼9.15 신청, 12월 말 지급<br>
- (하반기 소득분) 다음해 3.1∼3.15 신청, 6월말 정산(지급/환수)<br>
③ (신청방법)<br>
- 전자신청(ARS 전화 ☎1544-9944, 홈택스(PC, 모바일앱))<br>
- 신청대리* 서비스(장려금 상담센터(☎1566-3636)나 세무서로 전화하여 신청 요청)<br>
* 안내대상자 중 고령자, 장애인 등 전자신청을 어려워하는 경우

### 문의

국세상담센터(☎126)"""

#Vector Search Retriever
chroma_retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={"k":5},)

#BM25 Retriever
bm25_retriever = BM25Retriever.from_documents(documents=documents)
bm25_retriever.k = 5

#ensemble
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5],)

In [10]:
prompt_template = """당신은 지역사회에서 다양한 층의 사람들을 돕고 지원하는 사회복지사입니다.
    여러 사회문제에 대한 인식과 사회적 책임감을 가지고 있으며, 개별 개인의 상황을 정확히 파악하여 맞춤형 도움을 제공하는 데 주력하고 있습니다.
    사회복지사로서 아래 {context}를 기반으로 하여, 사람들의 질문에 최대한 상세하고 정확한 정보를 제공하기 위해 노력하고 있습니다.
    ==========
    #Context : 
    {context}
    
    ==========
    #Question: {query}
    """
    
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "query"])

In [ ]:
# con = """# 근로장려금

### 대상

근로·사업·종교인 소득이 있는 가구 중 소득·재산요건을 충족하는 가구
* '22.12.31. 현재 가구원(배우자·18세 미만 부양자녀·70세 이상 직계존속) 유무와 배우자의 총급여액 등에 따라 가구를 구분하며, 자녀장려금은 부양자녀가 있어야 합니다.

| 구분       | 설명 |
|------------|------|
| 단독가구   | 배우자, 부양자녀 및 부양 직계존속이 모두 없는 가구 |
| 홑벌이 가구 | 배우자, 18세 미만 부양자녀 또는 부양 직계존속이 있는 가구(단, 배우자의 총급여액 등이 300만 원 미만인 경우에 한함) |
| 맞벌이 가구 | 신청인과 배우자 각각의 총급여액 등이 300만 원 이상인 가구 |

- 재산요건: 2022년 6월 1일 기준 재산 합계액 2.4억 원 미만인 가구
- 소득요건: 가구 구성에 따라 2022년 총소득 기준금액이 아래 금액 미만인 가구

| 구분       | 단독가구     | 홑벌이 가구   | 맞벌이 가구   |
|------------|--------------|--------------|--------------|
| 총소득기준금액 | 2,200만 원 | 3,200만 원 | 3,800만 원 |
| 근로장려금 최대지급액 | 165만 원 | 285만 원 | 330만 원 |

※ 재산 합계액이 1억 7,000만 원 이상 2억 4,000만 원 미만일 경우 장려금의 50% 지급

### 내용

일은 하지만 소득이 적어 생활이 어려운 근로자, 사업자(전문직 제외) 또는 종교인 가구에 대하여 근로장려금을 지급함으로써 근로 또는 사업을 장려하고 실질소득을 지원

### 방법

① (원칙) 매년 5월 신청 → 9월 지급<br>
※ 기한 후 신청 시(6.1~11.30) 10% 감액 지급<br>
② (근로소득자) 당해연도 반기별 지급 선택 가능<br>
- (상반기 소득분) 9.1∼9.15 신청, 12월 말 지급<br>
- (하반기 소득분) 다음해 3.1∼3.15 신청, 6월말 정산(지급/환수)<br>
③ (신청방법)<br>
- 전자신청(ARS 전화 ☎1544-9944, 홈택스(PC, 모바일앱))<br>
- 신청대리* 서비스(장려금 상담센터(☎1566-3636)나 세무서로 전화하여 신청 요청)<br>
* 안내대상자 중 고령자, 장애인 등 전자신청을 어려워하는 경우

### 문의

국세상담센터(☎126)"""

In [12]:
# from analogical_prompt_test import lcel_prompt

# type(lcel_prompt(con, "소득이 적은데 무슨 도움을 받을 수 있을까요?"))
# type(con)
# # print(lcel_prompt(con, "소득이 적은데 무슨 도움을 받을 수 있을까요?"))

In [13]:
chain_type_kwargs = {"prompt": prompt}

In [14]:
#setup chain
chain = RetrievalQA.from_chain_type(
                    llm=llm,
                    chain_type="stuff",
                    retriever=ensemble_retriever,
                    chain_type_kwargs=chain_type_kwargs)

# print(ensemble_retriever.get_relevant_documents("대학생 국가장학금"))

###prompt 수정 필요
result = chain.run("소득이 적어 생활이 어려우면 어떡하죠?")
print(result)

[Document(page_content='국가장학금, 유형, 다자녀(세 자녀 이상)\n\n대상\n\n 구분  지원대상 \n\n 국가장학금 I유형  국내 대학에 재학 중인 학자금지원 8구간 이하 대학생 \n 국가장학금 II유형  학자금지원 9구간 이하 대학생 중 대학 자체 기준에 따라 선발 \n 다자녀 국가장학금(세자녀 이상)  국내 대학에 재학 중인 학자금지원 8구간 이하, 다자녀(자녀 셋 이상) 가구의 모든 대학생(미혼에 한함) \n\n 유형 및 다자녀 국가장학금은 성적(B0, 80100점 이상) 및 이수학점(직전학기 12학점 이상) 충족 필요\n\n 단, 기초·차상위 학생은 C0(70100점) 이상 적용, 학자금지원 13구간 학생은 C학점 경고제 2회 적용, 장애인 학생은 성적기준 미적용\n\n 신입생, 편입생, 재입학생은 첫 학기에 한해 성적기준 미적용\n\n내용\n\n유형, 다자녀  등록금 필수 경비(수업료)를 초과하지 않는 범위 내에서 차등 지원\n\n2023년 학자금 지원구간별 연간 국가장학금 최대 지원가능 금액 (단위  만 원)\n\n 구분  기초·차상위  1구간  2구간  3구간  4구간  5구간  6구간  7구간  8구간 \n\n 유형  700(둘째전액)  520  520  520  390  390  390  350  350 \n 다자녀(첫째,둘째)  700(둘째전액)  520  520  520  450  450  450  450  450 \n 다자녀(셋째 이상)  전액  전액  전액  전액  전액  전액  전액  전액  전액 \n\n유형  참여 대학의 자체 선발 기준(학자금 지원 9구간 이하)에 따라 장학금 차등 지원\n 단, 긴급하게 경제사정이 곤란하게 된 자 등 대학이 지원 필요성이 있다고 인정하는 경우 10구간도 지원 가능\n\n방법\n\n한국장학재단 누리집 및 모바일 앱에서 신청(httpswww.kosaf.go.kr)\n\n문의\n\n한국장학재단(15992000, www.kosaf.go.kr)\n\n알려드립니다\n\n학자금 지원구간이

In [ ]:
# Set up RetrievalQA chain using LCEL
chain = (
    {"context": ensemble_retriever, "question": prompt}
    | llm
    )

In [ ]:
# Run the chain with the modified prompt
result = chain.invoke({"context": con, "question": "소득이 적어 생활이 어려우면 어떡하죠?"})